In [12]:
CONFIG_FILE_PATH_PREFIX = 'cfg/'
CONFIG_TMP_FILE_PATH_PREFIX = 'cfg/tmp/'
CONFIG_BACKUP_FILE_PATH_PREFIX = 'cfg/backup/'

OUTPUT_FILE_PATH_PREFIX = 'output/'
OUTPUT_TMP_FILE_PATH_PREFIX = 'output/tmp/'

CONFIG_NUMBER = 1000

In [13]:
#!pip install Faker

In [14]:
import sys
import pyspark
from pyspark.sql import SparkSession, Row
from pyspark.sql.types import StructType,StructField, StringType
spark = SparkSession \
    .builder \
    .appName("Python Spark Customer Fact Generator") \
    .getOrCreate()

In [15]:
##Reading the config from file for max id
import os.path
idConfig = {
    "accountrecordid": 10000000,
    "defaultaccountrecordid": 10000000,
    "accountid": 10000000,
    "defaultaccountid": 10000000,
    "defaultcustomerid": 10000000,
    "customerid": 10000000,
}
print('-----------------')
for file in idConfig.keys():
    file_exist = os.path.exists(CONFIG_FILE_PATH_PREFIX+file)
    if (file_exist):
        print(file + ' is exist')
        print('begin loading' + file)
        realFile = open(CONFIG_FILE_PATH_PREFIX+file,"r")
        idConfig[file] = int(realFile.readlines()[0])
        realFile.close()
    else:
        if (file not in ["accountid", "defaultaccountid", "accountrecordid", "defaultaccountrecordid"]):
            print ('Please run CustomerFactGenerator first')
            break
        print(file + ' is not exist')
        print('begin to creating ' + file)
        realFile = open(CONFIG_FILE_PATH_PREFIX+file,"w+")
        realFile.seek(0)
        realFile.write(str(idConfig[file]))
        realFile.close()

-----------------
accountrecordid is exist
begin loadingaccountrecordid
defaultaccountrecordid is exist
begin loadingdefaultaccountrecordid
accountid is exist
begin loadingaccountid
-----------------
begin to creating defaultaccountid
defaultaccountid is exist
begin loadingdefaultaccountid
defaultcustomerid is exist
begin loadingdefaultcustomerid
customerid is exist
begin loadingcustomerid


In [16]:
print(idConfig)

{'accountrecordid': 10000000, 'defaultaccountrecordid': 10000000, 'accountid': 10000000, 'defaultaccountid': 10000000, 'defaultcustomerid': 10000000, 'customerid': 10001000}


In [25]:
import random
import faker
from datetime import datetime
import re
class AccountObject:
    AcountTypeDim = {
        1: "credit",
        2: "debit",
        3: "deposit",
        4: "saving"
    }
    StatusDim = {
        1: "active",
        2: "deactive",
        3: "temporarilylocked"
    }
    fake = faker.Faker()

    def __init__(self):
        self.accountrecordid = idConfig["accountrecordid"]
        idConfig["accountrecordid"] += 1
        self.accountid = idConfig["accountid"]
        idConfig["accountid"] += 1
        __accounttype = random.choice(list(AccountObject.AcountTypeDim.keys()))
        self.accounttype = AccountObject.AcountTypeDim[__accounttype]
        self.customerid = AccountObject.fake.random_int(min=idConfig['defaultcustomerid'], max=idConfig['customerid'])
        self.balance = random.randint(1, 10000)
        self.opendate = AccountObject.fake.date()
        self.closedate = ""
        self.initialamount = ""
        self.remainamount = ""
        self.totalamount = ""
        self.status = AccountObject.StatusDim[1]
        self.referenceinterestrate = ""
        self.cardinfo = AccountObject.fake.credit_card_full()
        self.lasttransactiondate =  AccountObject.fake.date_time_between(
                start_date=datetime.strptime(self.opendate, "%Y-%m-%d")).strftime("%Y-%m-%d, %H:%M:%S")
        self.lastactivedate = AccountObject.fake.date_time_between(
                start_date=datetime.strptime(self.opendate, "%Y-%m-%d")).strftime("%Y-%m-%d, %H:%M:%S")

    def toList(self):
        return [Row(
            self.accountrecordid,
            self.accountid,
            self.accounttype,
            self.customerid,
            self.balance,
            self.opendate,
            self.closedate,
            self.initialamount,
            self.remainamount,
            self.totalamount,
            self.status,
            self.referenceinterestrate,
            self.cardinfo,
            self.lasttransactiondate,
            self.lastactivedate

            )]


In [26]:
FactAccount = []
for x in range(CONFIG_NUMBER):
    Account = AccountObject().toList()
    FactAccount = FactAccount + Account

In [27]:
FactAccountColumns = StructType([
    StructField("accountrecordid", StringType(), True),
    StructField("accountid", StringType(), True),
    StructField("accounttype", StringType(), True),
    StructField("customerid", StringType(), True),
    StructField("balance", StringType(), True),
    StructField("opendate", StringType(), True),
    StructField("closedate", StringType(), True),
    StructField("initialamount", StringType(), True),
    StructField("remainamount", StringType(), True),
    StructField("totalamount", StringType(), True),
    StructField("status", StringType(), True),
    StructField("referenceinterestrate", StringType(), True),
    StructField("cardinfo", StringType(), True),
    StructField("lasttransactiondate", StringType(), True),
    StructField("lastactivedate", StringType(), True)
    ])

In [28]:
MainDf = spark.createDataFrame(data=FactAccount, schema = FactAccountColumns)

In [29]:
from datetime import datetime
now = datetime.now()
fileName = 'FactAccount ' + now.strftime("%Y-%m-%d %H-%M-%S-%f") 

In [30]:
print(fileName)

FactAccount 2022-11-22 11-22-24-935189


In [31]:
MainDf.write.mode('overwrite').option("header","true").csv(OUTPUT_TMP_FILE_PATH_PREFIX + fileName)

In [32]:
for file in idConfig.keys():
    print('begin to rewrite ' + file)
    realFile = open(CONFIG_TMP_FILE_PATH_PREFIX+file,"w+")
    realFile.seek(0)
    realFile.write(str(idConfig[file]))
    realFile.close()
    print('-----------------')

begin to rewrite accountrecordid
-----------------
begin to rewrite defaultaccountrecordid
-----------------
begin to rewrite accountid
-----------------
begin to rewrite defaultaccountid
-----------------
begin to rewrite defaultcustomerid
-----------------
begin to rewrite customerid
-----------------


In [33]:
import shutil
for file in idConfig.keys():
    shutil.move(CONFIG_FILE_PATH_PREFIX+file, CONFIG_BACKUP_FILE_PATH_PREFIX+file)
    shutil.move(CONFIG_TMP_FILE_PATH_PREFIX+file, CONFIG_FILE_PATH_PREFIX+file)
shutil.move(OUTPUT_TMP_FILE_PATH_PREFIX + fileName, OUTPUT_FILE_PATH_PREFIX+fileName)

'output/FactAccount 2022-11-22 11-22-24-935189'